In [34]:
import os
from functools import partial
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyro
import pyro.distributions as dist
from torch import nn
import seaborn as sns
from pyro.nn import PyroModule
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [50]:
#step-1 converting model to SCM 
def GroundTruthModel():
    alpha_1 = torch.tensor(0.75)
    beta_1 = torch.tensor(0.25)
    Nr = pyro.sample("Nr",dist.Categorical(torch.tensor([alpha_1,beta_1]))) 
    
    alpha_2 = torch.tensor(0.6)
    beta_2 = torch.tensor(0.4)
    Ns = pyro.sample("Ns",dist.Categorical(torch.tensor([alpha_2,beta_2])))
    
    alpha_3 = torch.tensor(0.)
    beta_3 = torch.tensor(1.)
    Nk = pyro.sample("Nk",dist.Normal(torch.tensor(alpha_3),torch.tensor(beta_3))) 
    
    Nr = Nr.type(torch.FloatTensor)
    Ns = Ns.type(torch.FloatTensor)
    Nk = Nk.type(torch.FloatTensor)
    
    gamma_1 =  pyro.sample("gamma_1",dist.Delta(Nr))
    gamma_2 =  pyro.sample("gamma_2",dist.Delta(Ns))
    gamma_3 =  pyro.sample("gamma_3",dist.Delta(Nk))

    
    R     = pyro.sample("R",dist.Delta(gamma_1))
    S     = pyro.sample("S",dist.Delta(gamma_2))
    K     = pyro.sample("K",dist.Delta(gamma_3))
    
    
    delta_1 = torch.tensor(0.)
    delta_2 = torch.tensor(1.)
    
    Gval  = K + 2.1* R + 3.3 * S + 0.5 * pyro.sample("g",dist.Normal(delta_1,delta_2))
    kappa_1 = torch.tensor(Gval)
    G     = pyro.sample("G",dist.Delta(kappa_1))
    
    Lval  = K + 5.8 * R + 0.7 * S + 0.1 * pyro.sample("l",dist.Normal(delta_1,delta_2))
    kappa_2 = torch.tensor(Lval)
    L     = pyro.sample("L",dist.Delta(kappa_2))
    
    Fval  = K + 2.3 * R + 1.0 * S + 0.3 * pyro.sample("f",dist.Normal(delta_1,delta_2))
    kappa_3 = torch.tensor(Fval)
    F     = pyro.sample("F",dist.Delta(kappa_3))
trace_handler = pyro.poutine.trace(GroundTruthModel)
samples = pd.DataFrame(columns=['R', 'S', 'K', 'G', 'L', 'F', 'p'])

In [52]:
#step-2 generate synthetic data from GTM
full_sample= []
for i in range(1000):
    trace = trace_handler.get_trace()
    R = trace.nodes['R']['value']
    S = trace.nodes['S']['value']
    K = trace.nodes['K']['value']
    G = trace.nodes['G']['value']
    L = trace.nodes['L']['value']
    F = trace.nodes['F']['value']
    # get prob of each combination
    log_prob = trace.log_prob_sum()
    p = np.exp(log_prob)
    samples = samples.append({'R': R, 'S': S, 'K': K, 'G': G, 'L':L, 'F': F, 'p': p}, ignore_index=True)
    full_sample.append(([R,S,G,L,K,F]))

samples.head()

C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.c

,R,S,K,G,L,F,p
0,tensor(1.),tensor(1.),tensor(0.7698),tensor(6.3037),tensor(7.3462),tensor(3.9295),tensor(0.0012)
1,tensor(1.),tensor(0.),tensor(-0.8265),tensor(1.9264),tensor(4.9697),tensor(1.5258),tensor(0.0011)
2,tensor(1.),tensor(1.),tensor(-0.1876),tensor(5.1110),tensor(6.4167),tensor(3.0110),tensor(0.0013)
3,tensor(0.),tensor(1.),tensor(1.5527),tensor(4.2713),tensor(2.2196),tensor(2.6095),tensor(0.0011)
4,tensor(1.),tensor(0.),tensor(-0.0741),tensor(2.7537),tensor(5.7093),tensor(2.4074),tensor(0.0011)


In [57]:
#step-3 GUIDE
def ProposedModel():
   
    alpha_1 = pyro.param('alpha_1', torch.tensor(0.75)) #, constraint=constraints.positive
    alpha_2 = pyro.param('alpha_2', torch.tensor(0.6))#
    alpha_3 = pyro.param('alpha_3', torch.tensor(0.))#
    
    beta_1 = pyro.param('beta_1', torch.tensor(0.25)) #
    beta_2 = pyro.param('beta_2', torch.tensor(0.4)) #
    beta_3 = pyro.param('beta_3', torch.tensor(1.)) #
    
    Nr = pyro.sample('Nr', dist.Categorical(torch.tensor([alpha_1,beta_1])))
    Ns = pyro.sample('Ns', dist.Categorical(torch.tensor([alpha_2,beta_2])))
    Nk = pyro.sample('Nk', dist.Normal(torch.tensor(alpha_3),torch.tensor(beta_3)))
    
    Nr = Nr.type(torch.FloatTensor)
    Ns = Ns.type(torch.FloatTensor)
    Nk = Nk.type(torch.FloatTensor)
    
    gamma_1 = pyro.sample('gamma_1',dist.Delta(Nr))
    gamma_2 = pyro.sample('gamma_2',dist.Delta(Nr))
    gamma_3 = pyro.sample('gamma_3',dist.Delta(Nr))
    
    
    R     = pyro.sample("R",dist.Delta(gamma_1))
    S     = pyro.sample("S",dist.Delta(gamma_2))
    K     = pyro.sample("K",dist.Delta(gamma_3))
    
    delta_1 = pyro.param('delta_1', torch.tensor(0.))#
    delta_2 = pyro.param('delta_2', torch.tensor(1.))#
    
    Gval  = K + alpha_1* R + beta_1 * S + gamma_1 * pyro.sample("g",dist.Normal(torch.tensor(delta_1),torch.tensor(delta_2)))
    
    G     = pyro.sample("G",dist.Delta(Gval))
    
    Lval  = K + alpha_2 * R + beta_2 * S + gamma_2 * pyro.sample("l",dist.Normal(torch.tensor(delta_1),torch.tensor(delta_2)))
    L     = pyro.sample("L",dist.Delta(Lval))
    
    Fval  = K + alpha_3 * R + beta_3 * S + gamma_3 * pyro.sample("f",dist.Normal(torch.tensor(delta_1),torch.tensor(delta_2)))
    F     = pyro.sample("F",dist.Delta(Lval))
trace_handler = pyro.poutine.trace(ProposedModel)
samples = pd.DataFrame(columns=['R', 'S', 'K', 'G', 'L', 'F', 'p'])

In [59]:
#Step 4 - train the ProposedModel on SD 
adam_params = {"lr": 0.0005}
optimizer = Adam(adam_params)
svi = SVI(GroundTruthModel, ProposedModel, optimizer, loss=Trace_ELBO())
n_steps = 2501
for step in range(n_steps):
    svi.step()

C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach()

In [61]:
#Printing samples from the PM model.
#Later we need to use this model to do step 5 through 8
from pyro.infer import Predictive


num_samples = 1000
predictive = Predictive(GroundTruthModel, guide=ProposedModel, num_samples=1000)
svi_samples = {k: v.reshape(num_samples).detach().cpu().numpy()
               for k, v in predictive().items()
               if k != "obs"}
print(svi_samples)

C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach()

{'Nr': array([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [69]:
#Step 5
def counterfactual(posterior):
    
    for _ in range(1):
        trace = posterior()
        #Not sure if we need to take all exogenous vars.?
        Nr = trace.nodes['Nr']['value']
        Ns = trace.nodes['Ns']['value']
        Nk = trace.nodes['Nk']['value']
       
    
    # Intervention Step
    intervention_model = pyro.do(ProposedModel, data = {"R": torch.tensor(0.),"S":torch.tensor(0.)})
    
    # Condition Step
    prediction_model = pyro.condition(intervention_model, data={"Nr": Nr, "Ns": Ns, "Nk": Nk})
    trace_prediction = pyro.poutine.trace(prediction_model)
    for _ in range(1):
        trace = trace_prediction.get_trace()
        K = trace.nodes['K']['value']
        return K


In [70]:
#What values do we need to condition the attributes L ,G on?? # Why counterfactual_samples generating only 0's?
conditoned_model = pyro.condition(ProposedModel, data = {"L": torch.tensor(0.), "G": torch.tensor(0.),"R":torch.tensor(0.),"S":torch.tensor(0.)})
posterior = pyro.infer.Importance(conditoned_model, num_samples=1000).run()
counterfactual_samples = [counterfactual(posterior) for _ in range(1000)]

C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\gowta\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach()

In [71]:
counterfactual_samples

[tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.),